In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords

In [2]:
url = 'https://finance.naver.com/news/mainnews.nhn'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
headline = []
for index, link in enumerate(soup.findAll("a", href=re.compile("^/news/news_read.nhn."))):
    if 'href' in link.attrs and link.get('title') != None:
        headline.append(link.get('title'))

In [4]:
headline

['라임 "기존 상환계획 이행 어렵다" 통보',
 "하루 10편 줄이고 3,500원 초특가에 풀어도 '썰렁'",
 "우한 '3차 전세기' 곧 출발…170여 명 귀환",
 "'봉준호·송강호·이미경' 블랙리스트들의 반란",
 '[공시]OCI, 지난해 영업손실 1807억..적자전환',
 '[공시]한전KPS, 지난해 영업익 1939억…전년比 1.3%↑',
 '[공시]CJ ENM, 덱스터 50억 유상증자 참여…지분 2% 확보',
 '라임운용, 투자자에 "계획대로 상환 이행 어렵다" 통보',
 '장 마감 후 기업공시[2월 11일]',
 '[표]아시아 주요 증시 동향(2월 11일)',
 '[마켓뷰] 2차전지株, 잇따른 호재로 상승장 이끌어',
 '공포에 \'산다\'…"믿을 건 삼성전자·하이닉스 뿐"',
 '장 마감 후 기업공시[2월 11일]',
 '"신종 코로나 `대박` 기회"…테마투자 부추기는 기생충들',
 '[Asia마감]돈 풀어 경기 부양…中증시 오름세',
 "현장 나타난 시진핑, '신종 코로나' 잡았나 [김현석의 월스트리트나우]",
 '라임 "기존 상환계획 이행 어렵다" 통보',
 "하루 10편 줄이고 3,500원 초특가에 풀어도 '썰렁'"]

In [5]:
# removing functuation
def remove_punctuation(text):
    import string
    # replacing the puntuations with no space,
    # which in effect deletes the punctuation marks
    translator = str.maketrans(' ', ' ', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

In [6]:
# extract the stop words
# extracting the stopwords
sw = stopwords.words()
print(f'Number of stopwords: {len(sw)}')
stopwords_ko = [line.replace('\n', '') for line in open('./meta/stopwords_ko', 'r', encoding='UTF8').readlines()]
print(f'Number of stopwords_ko: {len(stopwords_ko)}')
stopwords_total = stopwords_ko + sw
print(f'Number of stopwords_total: {len(stopwords_total)}')
# removing stopwords
def remove_stopwords(text):
    # removing the stop words and lowercasing the selected words
    text = [word for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

Number of stopwords: 6800
Number of stopwords_ko: 675
Number of stopwords_total: 7475


In [7]:
evaluation = pd.DataFrame()
evaluation['headline'] = headline
evaluation['tokenization'] = [re.sub(r'[^A-Za-z0-9가-힣一-龥]', ' ', i) for i in evaluation.headline]
evaluation['tokenization'] = evaluation['tokenization'].apply(remove_punctuation).apply(remove_stopwords)
evaluation['tokenization'] = [i.split(' ') for i in evaluation['tokenization']]

In [8]:
evaluation

,headline,tokenization
0,"라임 ""기존 상환계획 이행 어렵다"" 통보","[라임, 기존, 상환계획, 이행, 어렵다, 통보]"
1,"하루 10편 줄이고 3,500원 초특가에 풀어도 '썰렁'","[하루, 10편, 줄이고, 3, 500원, 초특가에, 풀어도, 썰렁]"
2,우한 '3차 전세기' 곧 출발…170여 명 귀환,"[우한, 3차, 전세기, 곧, 출발, 170여, 명, 귀환]"
3,'봉준호·송강호·이미경' 블랙리스트들의 반란,"[봉준호, 송강호, 이미경, 블랙리스트들의, 반란]"
4,"[공시]OCI, 지난해 영업손실 1807억..적자전환","[공시, OCI, 지난해, 영업손실, 1807억, 적자전환]"
5,"[공시]한전KPS, 지난해 영업익 1939억…전년比 1.3%↑","[공시, 한전KPS, 지난해, 영업익, 1939억, 전년比, 1, 3]"
6,"[공시]CJ ENM, 덱스터 50억 유상증자 참여…지분 2% 확보","[공시, CJ, ENM, 덱스터, 50억, 유상증자, 참여, 지분, 2, 확보]"
7,"라임운용, 투자자에 ""계획대로 상환 이행 어렵다"" 통보","[라임운용, 투자자에, 계획대로, 상환, 이행, 어렵다, 통보]"
8,장 마감 후 기업공시[2월 11일],"[장, 마감, 후, 기업공시, 2월, 11일]"
9,[표]아시아 주요 증시 동향(2월 11일),"[표, 아시아, 주요, 증시, 동향, 2월, 11일]"


In [9]:
negative = [line.replace('\n', '') for line in open('./meta/negative', 'r', encoding='UTF8').readlines()]
positive = [line.replace('\n', '') for line in open('./meta/positive', 'r', encoding='UTF8').readlines()]
sentimental = pd.read_csv('./meta/sentimental', sep='\t')

In [10]:
negative[:5], positive[:5]

(['가난', '가난뱅이', '가난살이', '가난살이하다', '가난설음'],
 ['(-;', '(^^)', '(^-^)', '(^^*', '(^_^)'])

In [11]:
sentimental.head(5)

,keyword,score
0,(-;,1
1,(;_;),-1
2,(^^),1
3,(^-^),1
4,(^^*,1


In [12]:
evaluation.head()

,headline,tokenization
0,"라임 ""기존 상환계획 이행 어렵다"" 통보","[라임, 기존, 상환계획, 이행, 어렵다, 통보]"
1,"하루 10편 줄이고 3,500원 초특가에 풀어도 '썰렁'","[하루, 10편, 줄이고, 3, 500원, 초특가에, 풀어도, 썰렁]"
2,우한 '3차 전세기' 곧 출발…170여 명 귀환,"[우한, 3차, 전세기, 곧, 출발, 170여, 명, 귀환]"
3,'봉준호·송강호·이미경' 블랙리스트들의 반란,"[봉준호, 송강호, 이미경, 블랙리스트들의, 반란]"
4,"[공시]OCI, 지난해 영업손실 1807억..적자전환","[공시, OCI, 지난해, 영업손실, 1807억, 적자전환]"


In [13]:
evaluation['positive_score'] = 0
evaluation['negative_score'] = 0

In [14]:
evaluation

,headline,tokenization,positive_score,negative_score
0,"라임 ""기존 상환계획 이행 어렵다"" 통보","[라임, 기존, 상환계획, 이행, 어렵다, 통보]",0,0
1,"하루 10편 줄이고 3,500원 초특가에 풀어도 '썰렁'","[하루, 10편, 줄이고, 3, 500원, 초특가에, 풀어도, 썰렁]",0,0
2,우한 '3차 전세기' 곧 출발…170여 명 귀환,"[우한, 3차, 전세기, 곧, 출발, 170여, 명, 귀환]",0,0
3,'봉준호·송강호·이미경' 블랙리스트들의 반란,"[봉준호, 송강호, 이미경, 블랙리스트들의, 반란]",0,0
4,"[공시]OCI, 지난해 영업손실 1807억..적자전환","[공시, OCI, 지난해, 영업손실, 1807억, 적자전환]",0,0
5,"[공시]한전KPS, 지난해 영업익 1939억…전년比 1.3%↑","[공시, 한전KPS, 지난해, 영업익, 1939억, 전년比, 1, 3]",0,0
6,"[공시]CJ ENM, 덱스터 50억 유상증자 참여…지분 2% 확보","[공시, CJ, ENM, 덱스터, 50억, 유상증자, 참여, 지분, 2, 확보]",0,0
7,"라임운용, 투자자에 ""계획대로 상환 이행 어렵다"" 통보","[라임운용, 투자자에, 계획대로, 상환, 이행, 어렵다, 통보]",0,0
8,장 마감 후 기업공시[2월 11일],"[장, 마감, 후, 기업공시, 2월, 11일]",0,0
9,[표]아시아 주요 증시 동향(2월 11일),"[표, 아시아, 주요, 증시, 동향, 2월, 11일]",0,0


In [15]:
positive_score, negative_score = [], []
p_dict = ['↑', '급상승', '상승', '올랐다', '호재']
n_dict = ['↓', '상장폐지', '뚝', '관리종목', '주의보', '낚시', '우려', '둔화', '공포', '사임', '어렵다', '썰렁']

for idx, row in evaluation.iterrows():
    p, n = 0, 0
    for pos in p_dict:
        if pos in row.headline:
            p += 1
            print(pos)
    for neg in n_dict:
        if neg in row.headline:
            n += 1
            print(neg)
    positive_score.append(p)
    negative_score.append(n)

↑
상승
공포


In [16]:
positive_score

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [17]:
negative_score

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]